In [140]:
import numpy as np
import random

class MABModel(object):
    p = np.zeros(1) # bernoulli arm probabilities
    curState = 0 # previous arm pulled
    def __init__(self, K):
        self.p =np.zeros([K,K])
        for i in range(K):
            for j in range(K):
                self.p[i,j] = random.uniform(0, 1)
        self.curState = random.randint(0,2)
    def setp(self,newp):
        self.p = newp
    def getArm(self, state):
        temp = 0
        if(random.uniform(0,1) < self.p[self.curState,state]):
            temp = 1
        self.curState = state
        return temp
    def getp(self):
        return self.p
x = MABModel(10)


In [132]:
import numpy as np
import copy


# p is the array of codependencies
# n is dimension of p
# start is the starting character of the sequence
# length is the number of arms to pull
# return a sequence of length number of arms that maximizes payoff
def bestSequence(n,p,start,length):
    seq = {} # dict mapping first arm to a tuple of best list starting with this arm and a total sum for that list
    # initialize the dictionary to all the possible final arms pulled
    for i in range(n):
        seq[i] = ([i],0)
    # loop through to generate arms of size length
    # only need to keep track of at most n sequences of pulls, one for each most recent arm pulled
    for i in range(length+1):
        tempseq = {} # temporary dictionary for the best lists of length one longer
        for ind in range(n):
            for recent in seq:
                l,val = seq[recent]
                # insert new list into dictionary if better than previous best
                if ind not in tempseq:
                    lind = copy.copy(l) 
                    lind.insert(0,ind) 
                    tempseq[ind] = (lind,val+p[ind,recent])
                else:
                    l2,val2 = tempseq[ind]
                    if val + p[ind,recent] > val2:
                        lind = copy.copy(l) 
                        lind.insert(0,ind) 
                        tempseq[ind] = (lind,val+p[ind,recent])                    
        seq = copy.copy(tempseq)
    # extract best list and value for given start
    a,b = seq[start]
    return a[1:],b

def countUnique(x):
    i = int(len(x)/2)
    k = x[i]
    count = 0
    while x[i+1] != k:
        count += 1
        i += 1
    return count+1

#### Testing
n = 10
p = np.random.rand(n,n)
average = 0
count = 0
for j in range(1000):
    p = np.random.rand(n,n)
    z,x = bestSequence(n,p,3,100)
    if countUnique(z) > 6:
        count += 1
    average += countUnique(z)
print(average/1000)
print(count)

2.117
6


In [164]:
# function to estimate the p array
def estimatep(num,n,x):
    curp =np.zeros([n,n])
    countp = np.zeros([n,n])
    prev = 0
    for i in range(1000):
        j = random.randint(0,n-1)
        curp[prev,j] += x.getArm(j)
        countp[prev,j] += 1
        prev = j
    return 2*curp/countp

def calculatep(totals, counts,n):
    tempcounts = np.ones([n,n])
    for c in range(n):
        for d in range(n): 
            if(counts[c,d] != 0):
                tempcounts[c,d] = counts[c,d]
    return totals/tempcounts
def creatematrix(n):
    newmatrix = np.zeros([n,n])
    for i in range(1,5):
        newmatrix[i-1,i] = 1
    newmatrix[4,0] = 1

    
n = 20
x = MABModel(n)
#x.setp(newmatrix)


seq,val = bestSequence(n,x.getp(),0,12000)
print(val)
print(seq[1000:1010])

curp = 5*np.ones([n,n])
countp = 5*np.ones([n,n])
total = 0
prev = 0
# loop through 700 runs
for i in range(1000):
    curp2 = calculatep(curp,countp,n) #current estimate of p
    seq, val = bestSequence(n,curp2,prev,12) #current best sequence of 10
    # randomize by epsilon
    for k in range(5):
        if random.randint(0,25000) < 1 or i <2 :
            seq[2*k] = random.randint(0,n-1)
            seq[2*k+1] = random.randint(0,n-1)
    # pull these 10 arms and update parameters
    for j in range(12):
        curVal = x.getArm(seq[j])
        total += curVal
        curp[prev,seq[j]] += curVal
        countp[prev,seq[j]] += 1
        prev = seq[j]
print(seq)
print(total)
print(x.getp())
print(calculatep(curp,countp,n))
print(np.count_nonzero(countp))
print(countp)

11777.3755369
[5, 4, 18, 5, 4, 18, 5, 4, 18, 5]
[11, 4, 18, 19, 2, 5, 4, 18, 19, 2, 5, 4, 16]
10992
[[ 0.23743698  0.07169787  0.62745347  0.16760583  0.81779601  0.41948921
   0.23412868  0.54111517  0.40464054  0.80037994  0.26673967  0.51853326
   0.17019814  0.64507814  0.91985583  0.44420848  0.72707002  0.46249674
   0.13395735  0.85319817]
 [ 0.8511353   0.23858696  0.80939003  0.91426659  0.13272981  0.09560659
   0.02080095  0.58574444  0.93295149  0.6485735   0.87400067  0.86432971
   0.97131721  0.91197519  0.68706545  0.64463788  0.44001693  0.06684526
   0.43786982  0.27859339]
 [ 0.94976042  0.13024789  0.78050688  0.51314688  0.82487452  0.98595269
   0.34391736  0.45883686  0.79799611  0.85462982  0.17046221  0.58779657
   0.98763637  0.69676089  0.29154179  0.99723235  0.94796653  0.71273216
   0.74910692  0.2479707 ]
 [ 0.96667719  0.38507147  0.67260196  0.02198396  0.71393286  0.17716257
   0.29108835  0.5489229   0.81450197  0.14060154  0.5092424   0.97307898
   0.